## Imports

In [1]:
import warnings
# warnings.filterwarnings("ignore")
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, UpSampling2D, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image 
from collections import Counter
import random

from sklearn.model_selection import train_test_split
from IPython.display import clear_output

import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader
from torchvision.transforms import functional as F

from torchvision.ops import box_iou
from torchvision.models.detection.faster_rcnn import FasterRCNN
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from tqdm import tqdm  # Import tqdm for progress bar

## Constants

In [2]:
url = "https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip"
filename = "FullIJCNN2013.zip"

data_dir = '/kaggle/working/FullIJCNN2013/FullIJCNN2013'
gt_file = '/kaggle/working/FullIJCNN2013/FullIJCNN2013/gt.txt'

annotations_file = "/kaggle/working/FullIJCNN2013/FullIJCNN2013/gt.txt"
images_dir       = "/kaggle/working/FullIJCNN2013/FullIJCNN2013"

class_number = 5
num_classes = 5 #4  

class_mapping = {
    0: "speed limit 20 (prohibitory)",
    1: "speed limit 30 (prohibitory)",
    2: "speed limit 50 (prohibitory)",
    3: "speed limit 60 (prohibitory)",
    4: "speed limit 70 (prohibitory)",
    5: "speed limit 80 (prohibitory)",
    6: "restriction ends 80 (other)",
    7: "speed limit 100 (prohibitory)",
    8: "speed limit 120 (prohibitory)",
    9: "no overtaking (prohibitory)",
    10: "no overtaking (trucks) (prohibitory)",
    11: "priority at next intersection (danger)",
    12: "priority road (other)",
    13: "give way (other)",
    14: "stop (other)",
    15: "no traffic both ways (prohibitory)",
    16: "no trucks (prohibitory)",
    17: "no entry (other)",
    18: "danger (danger)",
    19: "bend left (danger)",
    20: "bend right (danger)",
    21: "bend (danger)",
    22: "uneven road (danger)",
    23: "slippery road (danger)",
    24: "road narrows (danger)",
    25: "construction (danger)",
    26: "traffic signal (danger)",
    27: "pedestrian crossing (danger)",
    28: "school crossing (danger)",
    29: "cycles crossing (danger)",
    30: "snow (danger)",
    31: "animals (danger)",
    32: "restriction ends (other)",
    33: "go right (mandatory)",
    34: "go left (mandatory)",
    35: "go straight (mandatory)",
    36: "go right or straight (mandatory)",
    37: "go left or straight (mandatory)",
    38: "keep right (mandatory)",
    39: "keep left (mandatory)",
    40: "roundabout (mandatory)",
    41: "restriction ends (overtaking) (other)",
    42: "restriction ends (overtaking (trucks)) (other)"
}

## Plot Functions

In [3]:
def plot_annotations(image_path, annotations, image_number):
    image = Image.open(image_path)
    plt.imshow(image)
    plt.axis('off')

    plt.title(f"Image: {image_number}")

    for annotation in annotations:
        left, top, right, bottom, class_id = annotation
        class_name = class_mapping.get(class_id, "Unknown class")
        rect = plt.Rectangle((left, top), right-left, bottom-top, edgecolor='red', facecolor='none', linewidth=1)
        plt.gca().add_patch(rect)
        plt.text(left, top-20, class_name, color='red', fontsize=8, backgroundcolor='white', bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))

    plt.show()

def plot_samples(data_dir, gt_file, sample_count):
    annotations = load_annotations(gt_file)
    img_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.ppm')])

    random.shuffle(img_files)
    selected_files = img_files[:sample_count]

    for img_file in selected_files:
        img_path = os.path.join(data_dir, img_file)
        image_number = img_file.split('.')[0]
        if img_file in annotations:
            plot_annotations(img_path, annotations[img_file], image_number)

def plot_histogram(object_sizes):
    categories = list(object_sizes.keys())
    counts = list(object_sizes.values())

    plt.bar(categories, counts, color=['blue', 'orange', 'green'])
    plt.xlabel('Object Size')
    plt.ylabel('Count')
    plt.title('Object Size Distribution')
    plt.show()

def plot_class_histogram(data, title, xlabel, ylabel, color='purple'):
    categories = list(data.keys())
    counts = list(data.values())

    plt.figure(figsize=(12, 8))  # Increase figure size for better spacing
    plt.barh(categories, counts, color=color, height=0.8)  # Increase the height of bars for more spacing
    plt.xlabel(ylabel)  # Switch xlabel and ylabel for horizontal bar chart
    plt.ylabel(xlabel)
    plt.title(title)
    plt.gca().invert_yaxis()  # Invert y-axis to plot right to left
    plt.show()

## Utility functions

In [4]:
def get_class_value(class_number):
    return class_mapping.get(class_number, "Unknown class number")

def load_annotations(gt_file):
    annotations = {}
    with open(gt_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split(';')
            img_file = parts[0]
            left, top, right, bottom, class_id = map(int, parts[1:])
            if img_file not in annotations:
                annotations[img_file] = []
            annotations[img_file].append((left, top, right, bottom, class_id))
    return annotations

def categorize_objects(annotations):
    object_sizes = {'small': 0, 'medium': 0, 'large': 0}
    for img_file, objs in annotations.items():
        for obj in objs:
            left, top, right, bottom, class_id = obj
            width = right - left
            if width < 32:
                object_sizes['small'] += 1
            elif 32 <= width and width < 45:
                object_sizes['medium'] += 1
            else:
                object_sizes['large'] += 1
    return object_sizes

def count_objects_by_class(annotations):
    class_counts = Counter()
    for img_file, objs in annotations.items():
        for obj in objs:
            class_id = obj[4]
            class_name = class_mapping.get(class_id, "Unknown class")
            class_counts[class_name] += 1
    return class_counts

def collate_fn(batch):
    return tuple(zip(*batch))

def get_dataloaders(dataset, batch_size=4):
    train_dataset, val_dataset, test_dataset = dataset.split_dataset(val_ratio=0.1, test_ratio=0.2)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    return train_loader, val_loader, test_loader 

def get_model(num_classes):
    model = fasterrcnn_resnet50_fpn(weights="COCO_V1")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
        in_features, num_classes
    )
    return model

## Metric Functions

In [5]:
def calculate_iou(boxA, boxB):
    # Calculate the coordinates of the intersection rectangle
    inter_x1 = max(boxA[0], boxB[0])
    inter_y1 = max(boxA[1], boxB[1])
    inter_x2 = min(boxA[2], boxB[2])
    inter_y2 = min(boxA[3], boxB[3])

    # Compute the area of the intersection rectangle
    inter_width = max(0, inter_x2 - inter_x1)
    inter_height = max(0, inter_y2 - inter_y1)
    inter_area = inter_width * inter_height

    # Compute the area of both bounding boxes
    boxA_area = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxB_area = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # Compute the IoU
    iou = inter_area / float(boxA_area + boxB_area - inter_area)

    return iou

## Implementation

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [7]:
try:
    os.system(f"wget -O {filename} {url}")
    clear_output(wait=True)
    print(f"{filename} downloaded successfully.")
except Exception as e:
    print("An error occurred while downloading the dataset:", str(e))

FullIJCNN2013.zip downloaded successfully.


In [8]:
if os.path.exists(filename):
    try:
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("FullIJCNN2013")  
        print(f"{filename} extracted successfully.")
    except Exception as e:
        print("An error occurred while extracting the dataset:", str(e))
else:
    print(f"{filename} not found for extraction.")

FullIJCNN2013.zip extracted successfully.


In [9]:
# print(f"Class {class_number}: {get_class_value(class_number)}")

In [10]:
# plot_samples(data_dir, gt_file, sample_count=3)

In [11]:
annotations = load_annotations(gt_file)
object_sizes = categorize_objects(annotations)
# plot_histogram(object_sizes)

In [12]:
class_counts = count_objects_by_class(annotations)
# plot_class_histogram(class_counts, "Class Distribution", "Class", "Count", color='purple')

In [17]:
from torch.utils.data import Dataset, Subset

class FullIJCNN2013Dataset(Dataset):
    def __init__(self, annotations_file, images_dir, transform=None):
        self.annotations = self._load_annotations(annotations_file)
        self.images_dir = images_dir
        self.transform = transform
        self.label_to_id = {
            "prohibitory": 1,
            "danger": 2,
            "mandatory": 3,
            "other": 4
        }

    def _load_annotations(self, annotations_file):
        category_mapping = {
            "prohibitory": list(range(0, 6)) + list(range(7, 11)) + [15, 16],
            "danger": list(range(18, 32)) + [11],
            "mandatory": list(range(33, 41)),
            "other": [6, 12, 13, 14, 17, 32, 41, 42]
        }

        category_lookup = {}
        for category, ids in category_mapping.items():
            for cid in ids:
                category_lookup[cid] = category

        annotations = {}
        with open(annotations_file, "r") as file:
            for line in file:
                parts = line.strip().split(";")
                img_name = parts[0]
                bbox = tuple(map(int, parts[1:5]))
                class_id = int(parts[5])
                category = category_lookup.get(class_id, "other")

                if img_name not in annotations:
                    annotations[img_name] = []
                annotations[img_name].append({"bbox": bbox, "label": category})
        return annotations

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = list(self.annotations.keys())[idx]
        img_path = os.path.join(self.images_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        # # Convert image to a PyTorch tensor and normalize pixel values
        # image = torch.tensor(np.array(image), dtype=torch.float32) / 255.0
        # image = image.permute(2, 0, 1)  # Change from HWC to CHW format

        objects = self.annotations[img_name]
        bboxes = torch.tensor([obj["bbox"] for obj in objects], dtype=torch.float32)
        labels = torch.tensor(
            [self.label_to_id[obj["label"]] for obj in objects], dtype=torch.int64
        )

        target = {
            "boxes": bboxes,
            "labels": labels
        }

        if self.transform: image = self.transform(image)

        return image, target

    def split_dataset(self, val_ratio=0.1, test_ratio=0.2, random_state=123):
        indices = list(range(len(self)))
        temp_indices, test_indices = train_test_split(indices, test_size=test_ratio, random_state=random_state)
        train_indices, val_indices = train_test_split(temp_indices, test_size=val_ratio, random_state=random_state)
        
        train_dataset = Subset(self, train_indices)
        val_dataset   = Subset(self, val_indices)
        test_dataset  = Subset(self, test_indices)
        return train_dataset, val_dataset, test_dataset

In [18]:
def print_training_update(device, epoch, batch_idx, total_batches, loss): 
    print(f"Device: {device}, Epoch: {epoch}, Batch: {batch_idx}/{total_batches}, Loss: {loss:.4f}")

def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    running_loss = 0.0
    
    for i, (images, targets) in enumerate(data_loader):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        #xm.optimizer_step(optimizer)

        running_loss += losses.item()

        if i % 10 == 0:
            print_training_update(device, epoch, i, len(data_loader), losses.item())
        return running_loss / len(data_loader)

def evaluate(model, val_loader, device):
    model.eval()
    all_predictions = []
    all_targets = []    
    with torch.no_grad():
        for images, targets in val_loader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        predictions = model(images)
        
        all_predictions.extend(predictions)
        all_targets.extend(targets)

    return all_predictions, all_targets

In [22]:
from torchvision import transforms
import torchvision.transforms as T

transform = T.Compose([
    T.ToTensor(),  # Converts PIL.Image to torch.Tensor and normalizes to [0, 1]
])

# Normalization transform
mean = [0.485, 0.456, 0.406]  # Mean of ImageNet dataset
std = [0.229, 0.224, 0.225]   # Std of ImageNet dataset


transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images (adjust as needed)
    transforms.ToTensor(),         # Convert image to PyTorch tensor
    transforms.Normalize(mean, std)  # Apply normalization
])

dataset = FullIJCNN2013Dataset(annotations_file, images_dir, transform=transform)
train_loader, val_loader, test_loader = get_dataloaders(dataset)

# model = get_model(num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# num_epochs = 1 #10
# iou_scores = []
# for epoch in range(num_epochs):
#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
#     print(f"Training Loss: {train_loss:.4f}")
    
#     # val_loss = evaluate(model, val_loader, device)
#     predictions, targets = evaluate(model, val_loader, device)

#     for target, predict in zip(targets, predictions):
#         true_boxes = target["boxes"].cpu().numpy()
#         pred_boxes = predict["boxes"].cpu().numpy()

#     for tb, pb in zip(true_boxes, pred_boxes):
#             iou = calculate_iou(tb, pb)
#             iou_scores.append(iou)

    
#     print(f"Evaluated {len(predictions)} images., IoU socer: ,", iou)

# torch.save(model.state_dict(), "fasterrcnn_resnet50_cuda.pth")

In [24]:
!pip install -U pycocotools
!pip install -U faster-coco-eval
!pip install -U torchmetrics[detection]
clear_output(wait=True)

In [26]:
def train_faster_rcnn_staged(
    model, train_loader, val_loader, device,
    rpn_epochs, roi_epochs, fine_tune_epochs,
    rpn_patience=3, roi_patience=3, fine_tune_patience=3,
    lr=1e-3
):

####.....def early_stop........................................####
    def early_stop(metric_values, patience):
        if not metric_values:
            return False
    
        best_metric = max(metric_values)
        best_metric_epoch = len(metric_values) - 1 - metric_values[::-1].index(best_metric)
    
        if len(metric_values) - best_metric_epoch - 1 >= patience:
            recent_metrics = metric_values[best_metric_epoch + 1:]
            if all(m <= best_metric for m in recent_metrics[:patience]):
                return True
        return False
####.....end early_stop........................................####

####.....def compute_iou_loss........................................####
    def compute_iou_loss(val_loader):
        model.eval()
        total_iou = 0
        total_samples = 0
        with torch.no_grad():
            for images, targets in val_loader:
                images = list(img.to(device) for img in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                
                outputs = model(images)
                
                for output, target in zip(outputs, targets):
                    pred_boxes, target_boxes = output['boxes'], target['boxes']
                    if pred_boxes.numel() == 0 or target_boxes.numel() == 0:
                        print(f"Empty boxes found. Predicted: {pred_boxes.numel()}, Target: {target_boxes.numel()}")
                        continue
                    
                    ious = box_iou(pred_boxes, target_boxes)                   
                    if ious.numel() == 0: 
                        print(f"No valid IoUs found between predicted and target boxes.")
                        continue
                    
                    mean_iou = ious.mean().item()
                    if torch.isnan(torch.tensor(mean_iou)):
                        print(f"NaN IoU found. Predicted boxes: {pred_boxes}, Target boxes: {target_boxes}")
                        continue
                    
                    total_iou += mean_iou
                    total_samples += 1
        
        if total_samples == 0:
            print("No valid samples found in validation set for IoU computation.")
            return float('nan')  
        
        return total_iou / total_samples
####.....end compute_iou_loss........................................####

####.....def compute_map........................................####
    def compute_map(val_loader):
        model.eval()
        metric = MeanAveragePrecision()
        with torch.no_grad():
            for images, targets in val_loader:
                images = list(img.to(device) for img in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                outputs = model(images)
                metric.update(outputs, targets)
        return metric.compute()['map'].item()
####.....def compute_map........................................####

####.....def run_stage........................................####
    def run_stage(train_loader, val_loader, optimizer, num_epochs, patience, stage_name, eval_metric_fn):
        best_metric = 0
        metric_values = []
        for epoch in range(num_epochs):
            print(f"\n{stage_name} Training - Epoch {epoch + 1}/{num_epochs}")
            # Train for one epoch
            model.train()
            epoch_loss = 0
            for images, targets in tqdm(train_loader, desc="Batch", leave=True, ncols=100):
                images = list(img.to(device) for img in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                loss_dict = model(images, targets)
                total_loss = sum(loss for loss in loss_dict.values())
                
                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()
                
                epoch_loss += total_loss.item()
            
            print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {epoch_loss:.4f}")
            
            # Evaluate
            metric_value = eval_metric_fn(val_loader)
            metric_values.append(metric_value)
            print(f"Validation Metric ({stage_name}): {metric_value:.4f}")
            
            # Save the best model for this stage
            if metric_value > best_metric:
                best_metric = metric_value
                torch.save(model.state_dict(), f"best_{stage_name.lower()}_model.pth")
                print(f"Best model for {stage_name} saved.")
            
            # Check early stopping
            if early_stop(metric_values, patience):
                print(f"Early stopping triggered for {stage_name}.")
                break
####.....end run_stage........................................####
    

####.....train_faster_rcnn_staged........................................####
    print("Starting staged training...")
    # Stage 2: Train ROI Head
    for param in model.backbone.parameters():
        param.requires_grad = False
    for param in model.rpn.parameters():
        param.requires_grad = False
    for param in model.roi_heads.parameters():
        param.requires_grad = True
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    run_stage(train_loader, val_loader, optimizer, roi_epochs, roi_patience, "ROI Head", compute_iou_loss)
    
    # Stage 1: Train RPN
    for param in model.backbone.parameters():
        param.requires_grad = False
    for param in model.roi_heads.parameters():
        param.requires_grad = False
    for param in model.rpn.parameters():
        param.requires_grad = True
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    run_stage(train_loader, val_loader, optimizer, rpn_epochs, rpn_patience, "RPN", compute_iou_loss)
    
    # Stage 2: Train ROI Head
    for param in model.backbone.parameters():
        param.requires_grad = False
    for param in model.rpn.parameters():
        param.requires_grad = False
    for param in model.roi_heads.parameters():
        param.requires_grad = True
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    run_stage(train_loader, val_loader, optimizer, roi_epochs, roi_patience, "ROI Head", compute_map)
    
    # Stage 3: Fine-tune Entire Network
    for param in model.parameters():
        param.requires_grad = True
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr * 0.1)  # Lower learning rate for fine-tuning
    run_stage(train_loader, val_loader, optimizer, fine_tune_epochs, fine_tune_patience, "Fine-tuning", compute_map)

    print("Staged training complete.")

In [27]:
faster_rcnn_model = get_model(num_classes).to(device)
train_faster_rcnn_staged(
    faster_rcnn_model, train_loader, val_loader, device,
    rpn_epochs=100, roi_epochs=100, fine_tune_epochs=100,
    rpn_patience=5, roi_patience=5, fine_tune_patience=5,
    lr=1e-3
)

Starting staged training...

ROI Head Training - Epoch 1/100


Batch: 100%|██████████████████████████████████████████████████████| 133/133 [00:30<00:00,  4.43it/s]


Epoch 1/100 - Loss: 12314.6760
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found. 

Batch: 100%|██████████████████████████████████████████████████████| 133/133 [00:30<00:00,  4.41it/s]


Epoch 2/100 - Loss: 12298.1768
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found. 

...

Epoch 22/100 - Loss: 12313.0073
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found.

Batch: 100%|██████████████████████████████████████████████████████| 133/133 [00:30<00:00,  4.40it/s]


Epoch 23/100 - Loss: 12308.7770
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 12
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 4
Empty boxes found. Predicted: 0, Target: 8
Empty boxes found.

Batch:  95%|███████████████████████████████████████████████████▌  | 127/133 [00:29<00:01,  4.37it/s]


KeyboardInterrupt: 